In [3]:
#!/usr/bin/env python
# coding: utf-8

import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'module')))
from openTable import *
from AccesDB import *
from filepath import *
from preprocessing import preprocessing_text as pre

# import gensim
from gensim.models.ldamodel import LdaModel
from gensim.models import Phrases,TfidfModel
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel

from pickle import dump
from json import dumps
from datetime import datetime 
from re import sub
import warnings
warnings.filterwarnings('ignore')

# import spacy
from spacy.lang.id import Indonesian,stop_words
nlp = Indonesian()  # use directly
stopwords = stop_words.STOP_WORDS 
stopwords |= {"nya","jurusan","jurus","the","of"}

def preprocessing(text):
    text = pre.remove_tag(text) #Remove Tag
    text = pre.remove_whitespace(text) #Remove Whitespace
    text = pre.lower(text) #Lower
    text = pre.remove_link(text) #Remove Link
    text = pre.alphabet_only(text) #Get Alphabet
    text = sub(r'sobat pintar','',text) # sorry:(
    text = pre.remove_whitespace(text) #Remove Whitespace
    text = [token.text for token in nlp(text)] #Token
    text = pre.slang(text)
    text = [token.lemma_ for token in nlp(text) if token.lemma_ not in stopwords] #Lemma & stopword
    
    return text

def get_data(koneksi):
    data,status = open_table(koneksi,['entryId','content'],'BlogsEntry')
    
    return data

def get_best_topic(dictionary, corpus, texts, limit, start):
    coherence_values = []
    model_list = []
    
    for num_topics in range(start, limit):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=666)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    #get best model
    max_value = max(coherence_values)
    max_index = coherence_values.index(max_value)
    best_model = model_list[max_index]
        
    return best_model

def tfidf_keyword(corpus,dictionary,mylist):
    model = TfidfModel(corpus=corpus,id2word=dictionary)  # fit model
    model.save('../data/tfidf.h5')
    
    bow = [dictionary.doc2bow(mylist[i]) for i in range(len(mylist))]
    vector = [model[bow[i]] for i in range(len(mylist))]
    vector = [sorted(vector[i], key=lambda tup: tup[1],reverse=True) for i in range(len(vector))] #Sort
    
    feature = []
    result = []
    
    for i in vector:
        for j in range(len(i)):
            feature.append(dictionary[i[j][0]])

        result.append(feature[:150])
        feature = []
        
    return result

def make_corpus(data):
    #Make list of list
    mylist = []
    for i,j in data.iterrows():
        mylist.append(j.content)

    # Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
    bigram = Phrases(mylist, min_count=8)
    for idx in range(len(mylist)):
        for token in bigram[mylist[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                mylist[idx].append(token)

    # Create Dictionary
    dictionary = corpora.Dictionary(mylist)

    # Term Document Frequency
    corpus = [dictionary.doc2bow(text) for text in mylist]
    
    dump(corpus, open('../data/corpus_LDA.pkl', 'wb'))
    dictionary.save('../data/dictionary_LDA.gensim')
    
    return mylist,dictionary,corpus
    
def save_model(lda_model):
    #Save Model LDA
    lda_model.save('../data/lda.h5')

In [8]:
# def train():
prod_server,prod_koneksi = Connection()

dict_encoder = {}

try:
    #Get today Date
    date_end = datetime.today().strftime('%Y-%m-%d')

    #get data
    data = get_data(prod_koneksi)
    data = rename_column(data,{0:'entryId', 1:'content'})
#     data.content = data.content.apply(preprocessing)

#     #Save to Txt
#     with open('../data/train_news.txt', 'a+') as output:
#         output.write("Get Today Data Success, {} \n".format(date_end))
    print("Get today Date Success")

    prod_koneksi.close()
    prod_server.stop()
except Exception as e:
    print("Get today Date Failed",e)
#     with open('../data/train_news.txt', 'a+') as output:
#         output.write("Get Today Data Failed, {} \n".format(date_end))

    prod_koneksi.close()
    prod_server.stop()

Get today Date Success


In [10]:
data.content = data.content.apply(preprocessing)

In [11]:
data.values[0]

array([204420,
       list(['kakak', 'blog', 'membahas', 'tips', 'pembahasan', 'stan', 'postingan', 'kaka', 'bahas', 'rahasia', 'mudah', 'sbmtpn', 'sih', 'berlaku', 'deh', 'adik', 'masuk', 'perguruan', 'idaman', 'jalur', 'seleksi', 'masuk', 'perguruan', 'negeri', 'kali', 'disingkat', 'sbmptn', 'beruntung', 'dapatin', 'perguruan', 'snmptn', 'rahasia', 'paparkan', 'baunget', 'kud', 'baca', 'habis', 'deh', 'kaka', 'adik', 'yanng', 'baca', 'artikel', 'kakak', 'blog', 'milikku', 'salah', 'satunya', 'dr', 'orang', 'beruntung', 'lwat', 'undangan', 'khawatir', 'dikataka', 'gagal', 'rejekinya', 'indah', 'dunia', 'berhenti', 'berputar', 'lantaran', 'gagal', 'undangan', 'rute', 'adik', 'ikuti', 'tes', 'tertulis', 'tes', 'mandiri', 'still', 'keep', 'spirit', 'reach', 'your', 'dream', 'guys', 'kak', 'kaka', 'lulus', 'sbmptn', 'iya', 'yaaah', 'ketehe', 'hehehehahai', 'maksudnya', 'kakaak', 'dek', 'aye', 'peramal', 'dosalah', 'paranormal', 'syirik', 'kelulusan', 'klian', 'masuk', 'ptn', 'idaman', 'lw

In [12]:
#Make list of list
mylist = []
for i,j in data.iterrows():
    mylist.append(j.content)

# Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
bigram = Phrases(mylist, min_count=8)
for idx in range(len(mylist)):
    for token in bigram[mylist[idx]]:
        if '_' in token:
            print(token)
            # Token is a bigram, add to document.
            mylist[idx].append(token)

masuk_perguruan
jalur_seleksi
masuk_perguruan
salah_satunya
masuk_ptn
memiliki_peluang
seleksi_masuk
materi_pelajaran
ujian_tulis
masuk_ptn
kebanyakan_orang
materi_dipelajari
tes_masuk
perguruan_negeri
kerja_keras
oh_ya
menguasai_materi
membaca_buku
adik_adik
memahami_materi
baca_buku
les_privat
berkali_kali
les_privat
adik_adik
berkecil_hati
kemampuan_akademik
alat_tulis
mengikuti_tes
oh_ya
bangun_pagi
bangun_pagi
masuk_perguruan
ptn_favorit
snmptn_nilai
prestasi_akademik
semester_semester
peluang_lolos
rumah_sakit
non_akademik
non_akademik
non_akademik
menentukan_pilihan
anak_ipa
peluang_lolos
kakak_kakak
teman_temanmu
menarik_minatmu
patah_semangat
nilai_rapor
nilai_rapor
ratusan_ribu
peserta_sbmptn
ujian_masuk
ditengah_jalan
kerja_keras
jangka_pendek
bimbingan_belajar
try_out
try_out
try_out
try_out
mengikuti_ujian
berjam_jam
try_out
meja_belajar
penjelasan_guru
meja_belajar
memahami_materi
mind_maaf
penjelasan_guru
materi_pelajaran
penjelasan_guru
materi_pelajaran
penjelasan_guru


bertemu_matakuliah
wilayah_kota
hitung_hitungan
program_studi
perencanaan_wilayah
institut_teknologi
wilayah_kota
perencanaan_wilayah
universitas_brawijaya
malang_universitas
diponegoro_universitas
gadjah_mada
perencanaan_wilayah
berakreditasi_a
perguruan_negeri
sepuluh_nopember
universitas_sam
universitas_sebelas
maret_universitas
perguruan_swasta
institut_teknologi
malang_universitas
bandung_universitas
peluang_kerja
instansi_pemerintah
lingkungan_hidup
lulus_kuliah
konstruksi_bangunan
fresh_graduate
rumah_sakit
rumah_sakit
fakultas_teknik
ditingkat_perguruan
jenjang_pendidikan
ahli_madya
bergelar_sarjana
bergelar_sarjana
jenjang_s
kesehatan_masyarakat
kesehatan_masyarakat
memiliki_kecakapan
rumah_sakit
matematika_fisika
buku_buku
berbahasa_inggris
alumni_teknik
kemenkes_semarang
universitas_airlangga
oh_ya
jenjang_s
pendidikan_dokter
fakultas_kedokteran
rekayasa_hayati
rekayasa_hayati
teknologi_hayati
for_your
nonton_film
institute_technology
bidang_ilmu
institut_teknologi
rekayasa_

baca_buku
kegiatan_ekstrakurikuler
teman_temanmu
buku_buku
kerja_keras
guru_bp
orang_tua
teman_teman
kegiatan_ekstrakurikuler
artificial_intelligence
teman_temanmu
masuk_ptn
orang_tua
memiliki_kelebihan
ptn_pts
kualitas_pendidikan
negeri_swasta
lembaga_pendidikan
persaingan_masuknya
sesuai_kebutuhan
snmptn_sbmptn
penerimaan_mahasiswa
penerimaan_mahasiswa
kualitas_pendidikan
tenaga_pengajar
dosen_dosen
penerimaan_mahasiswa
pembiayaan_perkuliahan
memperoleh_beasiswa
pembiayaan_perkuliahan
faktor_faktor
peluang_kerja
didunia_kerja
berakreditasi_a
memiliki_peluang
ptn_pts
perusahaan_swasta
memiliki_peluang
didunia_kerja
peluang_kerja
bersikap_terbuka
seiring_perkembangan
sudut_pandang
kegiatan_kemahasiswaan
universitas_gadjah
mada_institut
teknologi_bandung
institut_pertanian
bogor_universitas
institut_teknologi
sepuluh_nopember
universitas_diponegoro
universitas_airlangga
universitas_brawijaya
universitas_hasanuddin
universitas_negeri
perguruan_terbaik
alih_alih
universitas_gadjah
mada_un

membuatmu_nyaman
salah_faktor
sungguh_sungguh
orang_tua
tolok_ukur
karya_karya
sesuai_untukmu
bersikap_sopan
bahasa_asing
seni_kriya
sia_sia
berinteraksi_orang
membuatmu_nyaman
bersikap_positif
orang_orang
bersikap_positif
anak_magang
job_desc
media_sosial
didunia_maya
media_sosial
teman_temanmu
magang_perusahaan
alih_alih
didunia_maya
bersikap_sopan
terima_kasih
sekolah_dasar
sebelah_mata
sekolah_dasar
ayo_lihat
bahasa_indonesia
peluang_kerja
sekolah_dasar
sma_ma
materi_pelajaran
bahasa_inggris
seni_tari
seni_musik
semester_semester
bahasa_inggris
semester_semester
belajar_mengajar
matakuliah_matakuliah
sekolah_dasar
lho_ya
tenaga_pendidik
bimbingan_konseling
anak_anak
sekolah_dasar
sekolah_dasar
percaya_deh
anak_anak
sekolah_dasar
universitas_negeri
jalur_sbmptn
universitas_negeri
bersaing_ketat
sekolah_dasar
memiliki_peluang
sekolah_dasar
tenaga_pendidik
pegawai_negeri
orang_tua
sekolah_dasar
sekolah_dasar
prospek_kerja
kota_kota
percaya_coba
program_sarjana
sekolah_dasar
alih_alih


contoh_sederhananya
perusahaan_swasta
negeri_surabaya
memilih_kuliah
tidur_nyenyak
lulus_ipk
negeri_surabaya
menentukan_pilihan
negeri_surabaya
calon_mahasiswa
memilih_kuliah
bakat_minat
menentukan_pilihan
bercita_cita
puluhan_juta
garuda_indonesia
mencapai_rupiah
rupiah_rupiah
fresh_graduate
rupiah_juta
mencapai_rupiah
lulus_kuliah
rupiah_rupiah
berpenghasilan_rupiah
rupiah_juta
human_resource
human_resource
human_resource
puluhan_juta
cita_citamu
rumah_sakit
rumah_sakit
berkisar_rupiah
dokter_spesialis
dokter_spesialis
rumah_sakit
dokter_spesialis
rumah_sakit
teknologi_informasi
dunia_kerja
teknologi_informasi
perangkat_lunak
sistem_informasi
software_engineer
software_developer
rupiah_rupiah
rupiah_rupiah
rupiah_rupiah
rupiah_rupiah
berkembang_pesat
negara_negara
lulusan_teknik
teknik_arsitektur
project_manager
rupiah_rupiah
fresh_graduate
rupiah_juta
rupiah_juta
berkecil_hati
amerika_serikat
science_technology
tenaga_kerja
negara_maju
menarik_minatmu
bidang_bidang
kimia_fisika
diba

...

memperbesar_peluang
anak_ipa
lolos_sbmptn
anak_ips
dikelas_ips
anak_ipa
jalur_snmptn
dikelas_ips
anak_ipa
kelas_ips
kuliah_dijurusan
jalur_snmptn
universitas_diponegoro
universitas_airlangga
daya_tampung
perencanaan_wilayah
rumpun_saintek
anak_ips
perencanaan_wilayah
mengikuti_snmptn
universitas_diponegoro
ipa_ips
memiliki_minat
ilmu_aktuaria
rumpun_saintek
does_it
universitas_gadjah
ilmu_aktuaria
dikelas_ips
jalur_snmptn
cita_citamu
daya_tampung
anak_ips
universitas_gadjah
rumpun_saintek
jalur_snmptn
pembangunan_wilayah
pembangunan_wilayah
jalur_snmptn
universitas_gadjah
pembangunan_wilayah
kelas_ips
materi_perkuliahan
menerima_mahasiswa
kelas_ipa
jalur_snmptn
anak_ipa
menerima_mahasiswa
kelas_ips
mengikuti_snmptn
daya_tampung
jalur_snmptn
budidaya_perairan
budidaya_perairan
menerima_mahasiswa
kelas_ipa
anak_ips
mengikuti_snmptn
daya_tampungnya
anak_ips
jalur_snmptn
universitas_airlangga
daya_tampungnya
gelar_sarjana
dikelas_ips
mengikuti_seleksi
anak_ipa
peluangmu_lolos
patah_semanga

sudut_pandang
sudut_pandang
memiliki_akreditasi
lapangan_kerja
sarjana_sains
salah_bentuk
mudah_dipahami
akreditasi_a
ban_perusahaan
universitas_gadjah
jalur_masuk
snmptn_sbmptn
ujian_tulis
daya_tampung
jalur_sbmptn
universitas_brawijaya
daya_tampung
jalur_sbmptn
seleksi_masuk
universitas_brawijaya
universitas_airlangga
sains_teknologi
bidang_keahlian
statistika_matematika
jalur_masuk
snmptn_sbmptn
daya_tampung
sbmptn_kursi
universitas_diponegoro
matakuliah_matakuliah
snmptn_sbmptn
jalur_masuk
jalur_sbmptn
daya_tampung
sepi_peminat
universitas_hasanuddin
jalur_masuk
calon_mahasiswa
snmptn_sbmptn
jalur_sbmptn
daya_tampung
universitas_padjadjaran
quality_control
jalur_masuk
snmptn_sbmptn
institut_pertanian
persaingan_masuk
terbaik_indonesia
sulit_ditembus
jalur_masuk
snmptn_sbmptn
jalur_sbmptn
daya_tampung
institut_teknologi
sepuluh_nopember
matematika_statistika
terbagi_kedalam
matematika_statistika
sains_data
daya_tampung
jalur_masuk
patut_dipertimbangkan
kampus_kampus
akreditasi_b
pro

memilih_kuliah
know_what
orang_tua
kuliah_dijurusan
orang_tua
bakat_minat
terbaik_indonesia
qs_world
teknologi_industri
manajemen_hutan
teknologi_hasil
konservasi_sumberdaya
seni_pertunjukan
isi_yogyakarta
patut_dipertimbangkan
fakultas_seni
isi_yogyakarta
seni_tari
seni_pertunjukan
seni_pertunjukan
jawa_barat
terbaik_indonesia
fakultas_seni
rupa_desain
seni_rupa
desain_komunikasi
desain_interior
desain_produk
when_it
fakultas_ekonomi
ilmu_ekonomi
ekonomi_islam
sesuai_untukmu
minat_pintar
akuntansi_keuangan
bahasa_kebudayaan
sastra_jawa
sastra_indonesia
sastra_inggris
sastra_perancis
fakultas_ilmu
qs_world
bisnis_manajemen
program_studi
ilmu_ekonomi
teknik_kimia
layak_dipertimbangkan
teknik_kimia
teknologi_pangan
teknik_kimia
kampus_pintar
computer_science
information_system
teknik_elektro
teknologi_informasi
teknik_elektro
teknik_elektro
tenaga_listrik
fakultas_teknik
teknik_mesin
patut_dipertimbangkan
seleksi_masuk
proses_seleksi
masuk_ptn
jalur_snmptn
jalur_sbmptn
jalur_seleksi
jalu

rumpun_saintek
skor_utbk
pendidikan_dokter
ilmu_komputer
teknik_industri
peluang_lolos
rumpun_soshum
sulit_ditembus
hubungan_internasional
ilmu_ekonomi
ilmu_ekonomi
nilai_utbk
lolos_sbmptn
sulit_ditembus
skor_utbk
peluang_lolos
try_out
peluang_lolos
dijadwalkan_april
universitas_indonesia
pilihan_kuliah
pkn_stan
sekolah_kedinasan
ikatan_dinas
ikatan_dinas
seleksi_masuk
pkn_stan
ujian_tulis
ujian_masuk
pkn_stan
tes_potensi
ambang_batas
ambang_batas
ambang_batas
ambang_batas
bahasa_inggris
ambang_batas
don_t
seleksi_masuk
pkn_stan
ambang_batas
hasil_ujian
ambang_batas
lolos_seleksi
ambang_batas
ambang_batas
pkn_stan
mengikuti_ujian
sistem_gugur
sistem_gugur
ujian_masuk
pkn_stan
pkn_stan
sistem_gugur
pkn_stan
sistem_gugur
pkn_stan
lulus_ipk
sistem_gugur
pkn_stan
pkn_stan
sistem_gugur
menghadapi_ujian
there_s
no_need
dosen_dosen
pkn_stan
sungguh_sungguh
nilai_minimal
pkn_stan
kampus_kampus
sistem_gugur
pkn_stan
sistem_gugur
pkn_stan
ya_sobat
impian_cita
pernahkah_sobat
berkembang_pesat
tek

pilihan_kuliah
jalur_snmptn
seleksi_mandiri
daya_tampung
biaya_hidup
jalur_seleksi
universitas_indonesia
daya_tampung
rumpun_soshum
snmptn_kursi
sbmptn_kursi
ya_sobat
universitas_indonesia
program_studi
universitas_islam
memiliki_akreditasi
universitas_indonesia
in_fact
sastra_asing
daya_tampung
terakreditasi_a
snmptn_kursi
sbmptn_kursi
universitas_gadjah
dijurusan_belajar
terakreditasi_a
daya_tampung
rumpun_saintek
daya_tampungnya
sbmptn_daya
teknik_nuklir
universitas_gadjah
fisika_matematika
teknik_nuklir
ptn_pts
it_s
lolos_seleksi
akreditasi_a
snmptn_kursi
pilihan_kuliahmu
berkali_kali
take_your
diakhir_pekan
tidur_siang
tidur_siang
diakhir_pekan
memejamkan_mata
diakhir_pekan
tidur_siang
berjam_jam
tugas_tugas
hang_out
teman_teman
jalan_jalan
orang_orang
uang_saku
menu_makanan
ya_sobat
teman_teman
makanan_sehat
diakhir_pekan
nonton_film
teman_teman
tugas_tugas
main_game
diakhir_pekan
main_game
tugas_tugas
kerajinan_tangan
diakhir_pekan
sulit_berkonsentrasi
ilmu_gizi
ilmu_gizi
rumah_

In [54]:
# Create Dictionary
dictionary = corpora.Dictionary(result)

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in result]

In [61]:
dictionary[2]

'adik_adik'

In [58]:
corpus[0][0]

(0, 1)

In [57]:
result[0]

['sbmptn',
 'allah',
 'pembahasan',
 'rahasia',
 'dikasi',
 'tes',
 'kaka',
 'bab',
 'berdo',
 'video',
 'adik',
 'lulus',
 'bagus',
 'panitia',
 'tips',
 'adiik',
 'aye',
 'loh',
 'optimal',
 'maksud',
 'serius',
 'materi',
 'belajarnya',
 'akang',
 'andaikan',
 'dapatin',
 'kakaak',
 'kakakk',
 'frekuensi',
 'muslim',
 'privat',
 'belajar',
 'sbmtpn',
 'sih',
 'deh',
 'usahakan',
 'strategi',
 'buku',
 'minder',
 'peserta',
 'kakak',
 'mudah',
 'percaya',
 'kak',
 'tesnya',
 'masuk_perguruan',
 'nilai',
 'benaran',
 'besoknya',
 'dech',
 'hehehe',
 'impuls',
 'insya',
 'kakaaaak',
 'kalao',
 'noleh',
 'penjelasannya',
 'saranin',
 'satuu',
 'sbmtptn',
 'sholat',
 'waaah',
 'yaaah',
 'ygg',
 'idaman',
 'sulit',
 'h',
 'hasilnya',
 'ptn',
 'belas',
 'kalu',
 'nan',
 'tertulis',
 'ruangan',
 'murah',
 'nomor',
 'pelajaran',
 'kuasai',
 'masuk',
 'selesai',
 'otak',
 'blog',
 'beruntung',
 'pembelajaran',
 'baca',
 'bahasan',
 'kostum',
 'lantaran',
 'dr',
 'habis',
 'multimedia',
 'iya'

In [ ]:
try:
    start=3
    limit=51
    best_model = get_best_topic(dictionary, corpus=corpus, texts=result, start=start, limit=limit)

    save_model(best_model)
    print("Train LDA Success")
except Exception as e:
    print("Train LDA Failed",e)

#     prod_koneksi.close()
#     prod_server.stop()

# train()

In [22]:
from time import process_time

In [23]:
t = process_time()
#do some stuff
train()
elapsed_time = process_time() - t
print(elapsed_time)

Get today Date Success
Make Dictionary and Corpus Success
get keyword Success
Train LDA Success
363.515625


In [66]:
start=3
limit=51
#     best_model = get_best_topic(dictionary, corpus=corpus, texts=result, start=start, limit=limit)

coherence_values = []
model_list = []

for num_topics in range(start, limit):
    model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=666)
    model_list.append(model)
    coherencemodel = CoherenceModel(model=model, texts=result, dictionary=dictionary, coherence='c_v')
    coherence_values.append(coherencemodel.get_coherence())

#get best model
max_value = max(coherence_values)
max_index = coherence_values.index(max_value)
best_model = model_list[max_index]

In [67]:
max_index

26

In [68]:
coherence_values

[0.3089600551146667,
 0.34370863912714655,
 0.338183949757659,
 0.35163041683105223,
 0.35278655460486846,
 0.3592902607003623,
 0.35813892753752496,
 0.34125105657648613,
 0.3532630382278977,
 0.3379855007772965,
 0.3451641630483382,
 0.3547692598157431,
 0.34861842152354183,
 0.35574770511541565,
 0.3476436953480907,
 0.35121524895708073,
 0.34611503171661906,
 0.35002496767198654,
 0.3533636151613512,
 0.33401077435273685,
 0.34812245067384096,
 0.34896986576006955,
 0.3540586432831131,
 0.3485249313805451,
 0.3523635165577159,
 0.3537086169621161,
 0.372039410355357,
 0.3425107146358034,
 0.34907747941072764,
 0.3583775940874775,
 0.36110156480732986,
 0.3495115783492601,
 0.34379032804439924,
 0.3551003311371466,
 0.35263016152039645,
 0.3487445269170682,
 0.34357551307604783,
 0.3557582713063938,
 0.35024068988443036,
 0.3562380887716826,
 0.3624562322768563,
 0.34982839400239435,
 0.3561783839992994,
 0.3522213616647896,
 0.34706066329056423,
 0.3564660125102305,
 0.339100017707

In [53]:
coherence_values

[0.35319576058808816,
 0.37361238970825084,
 0.38764631240018754,
 0.392688660038308,
 0.3908433458604996,
 0.39798377379817457,
 0.398955769964878,
 0.38202428596178895,
 0.3890094632350358,
 0.3846034601292296,
 0.38087743407441355,
 0.3863738683426908,
 0.38604203611250426,
 0.38877137688394914,
 0.3927377176304404,
 0.39383963353275164,
 0.3967601586509492,
 0.38959623311533237,
 0.39127412905866715,
 0.3867122746522514,
 0.38602460479942524,
 0.38194128616755635,
 0.3900310383989224,
 0.3876964995959675,
 0.38451127528536594,
 0.38469077994185386,
 0.3850633719526269,
 0.382291211251281,
 0.37911080696588667,
 0.3872653945334731,
 0.39088249573593925,
 0.38479149241229105,
 0.38499119243055324,
 0.3846915315192645,
 0.3880475068591031,
 0.38320485883165706,
 0.38766044327486004,
 0.3893938642010524,
 0.3908262079091045,
 0.38478805156174545,
 0.38415350765074235,
 0.38962578194301334,
 0.3850533279822327,
 0.38904050874148194,
 0.38707439958403217,
 0.37965200922778924,
 0.3804581

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'module')))
from openTable import *
from AccesDB import *
from filepath import *
from preprocessing import preprocessing_text as pre

# import gensim
from gensim.models.ldamodel import LdaModel
from gensim.models import Phrases,TfidfModel
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel

from pickle import dump
from json import dumps
from datetime import datetime 
from re import sub
import warnings
warnings.filterwarnings('ignore')

# import spacy
from spacy.lang.id import Indonesian,stop_words
nlp = Indonesian()  # use directly
stopwords = stop_words.STOP_WORDS 
stopwords |= {"nya","jurusan","jurus","the","of"}

def preprocessing(text):
    text = pre.remove_tag(text) #Remove Tag
    text = pre.remove_whitespace(text) #Remove Whitespace
    text = pre.lower(text) #Lower
    text = pre.remove_link(text) #Remove Link
    text = pre.alphabet_only(text) #Get Alphabet
    text = sub(r'sobat pintar','',text) # sorry:(
    text = pre.remove_whitespace(text) #Remove Whitespace
    text = [token.text for token in nlp(text)] #Token
    text = pre.slang(text)
    text = [token.lemma_ for token in nlp(text) if token.lemma_ not in stopwords] #Lemma & stopword
    
    return text

def get_data(koneksi):
    data,status = open_table(koneksi,['entryId','content'],'BlogsEntry')
    
    return data

def get_best_topic(dictionary, corpus, texts, limit, start):
    coherence_values = []
    model_list = []
    
    for num_topics in range(start, limit):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=666)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    #get best model
    max_value = max(coherence_values)
    max_index = coherence_values.index(max_value)
    best_model = model_list[max_index]
        
    return best_model

def tfidf_keyword(corpus,dictionary,mylist):
    model = TfidfModel(corpus=corpus,id2word=dictionary)  # fit model
    model.save('../data/tfidf.h5')
    
    bow = [dictionary.doc2bow(mylist[i]) for i in range(len(mylist))]
    vector = [model[bow[i]] for i in range(len(mylist))]
    vector = [sorted(vector[i], key=lambda tup: tup[1],reverse=True) for i in range(len(vector))] #Sort
    
    feature = []
    result = []
    
    for i in vector:
        for j in range(len(i)):
            feature.append(dictionary[i[j][0]])

        result.append(feature[:150])
        feature = []
        
    return result

def make_corpus(data):
    #Make list of list
    mylist = []
    for i,j in data.iterrows():
        mylist.append(j.content)

    # Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
    bigram = Phrases(mylist, min_count=8)
    for idx in range(len(mylist)):
        for token in bigram[mylist[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                mylist[idx].append(token)

    # Create Dictionary
    dictionary = corpora.Dictionary(mylist)

    # Term Document Frequency
    corpus = [dictionary.doc2bow(text) for text in mylist]
    
    dump(corpus, open('../data/corpus_LDA.pkl', 'wb'))
    dictionary.save('../data/dictionary_LDA.gensim')
    
    return mylist,dictionary,corpus
    
# def save_model(lda_model):
#     #Save Model LDA
#     lda_model.save('../data/lda.h5')
    
def train():
    prod_server,prod_koneksi = Connection()

    dict_encoder = {}
    
    try:
        #Get today Date
        date_end = datetime.today().strftime('%Y-%m-%d')

        #get data
        data = get_data(prod_koneksi)
        data = rename_column(data,{0:'entryId', 1:'content'})
        data.content = data.content.apply(preprocessing)
        
        #Save to Txt
        with open('../data/train_news.txt', 'a+') as output:
            output.write("Get Today Data Success, {} \n".format(date_end))
        print("Get today Date Success")
        
        prod_koneksi.close()
        prod_server.stop()
    except Exception as e:
        print("Get today Date Failed",e)
        with open('../data/train_news.txt', 'a+') as output:
            output.write("Get Today Data Failed, {} \n".format(date_end))
            
        prod_koneksi.close()
        prod_server.stop()
    
    try:
        #make corpus
        mylist,dictionary,corpus = make_corpus(data)
        
        #make dict encode
        for i,j in zip(range(len(mylist)),data.entryId.tolist()):
            dict_encoder[i] = j
            
        with open('../data/dict_encoder.txt', 'w') as file:
            file.write(dumps(dict_encoder)) # use `json.loads` to do the reverse
        print("Make Dictionary and Corpus Success")

    except Exception as e:
        print("Make Dictionary and Corpus Failed")
        
    try:
        #TF-IDF
        mylist = tfidf_keyword(corpus,dictionary,mylist)
        print("get keyword Success")
    except Exception as e:
        print("get keyword Failed",e)
        
    try:
        #make corpus
        mylist,dictionary,corpus = make_corpus(data)
        
#         #make dict encode
#         for i,j in zip(range(len(mylist)),data.entryId.tolist()):
#             dict_encoder[i] = j
            
#         with open('../data/dict_encoder.txt', 'w') as file:
#             file.write(dumps(dict_encoder)) # use `json.loads` to do the reverse
        print("Make Dictionary and Corpus Success")

    except Exception as e:
        print("Make Dictionary and Corpus Failed")
        
#     try:
#         start=3
#         limit=51
#         best_model = get_best_topic(dictionary, corpus=corpus, texts=mylist, start=start, limit=limit)
        
# #         save_model(best_model)
#         print("Train LDA Success")
#     except Exception as e:
#         print("Train LDA Failed",e)
        
#     prod_koneksi.close()
#     prod_server.stop()
    
# train()